In [1]:
# needed to import dataset from google drive into colab
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
cd gdrive

/content/gdrive


In [3]:
cd MyDrive

/content/gdrive/MyDrive


In [5]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-jhwg4m6r
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-jhwg4m6r
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.5 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369498 sha256=5e6eaba48459a18f63d02c142a49cd7f6ea4faa941c73bf7140bc388e197a0a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-z_ika399/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [6]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import clip
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [7]:
cnn_data_folder = 'cnn_dataset'

In [8]:
# Step 7: Load and Preprocess Images for Nature Classification
class ImageDataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.transform = transform
        self.images, self.labels = self.load_images()

    def load_images(self):
        images = []
        labels = []
        for class_label, class_name in enumerate(os.listdir(self.data_folder)):
            class_folder = os.path.join(self.data_folder, class_name)
            if not os.path.isdir(class_folder):
                continue
            for file_name in os.listdir(class_folder):
                file_path = os.path.join(class_folder, file_name)
                image = cv2.imread(file_path)
                image = cv2.resize(image, (224, 224))  # Resize image to fit CLIP model input shape
                label = class_label
                images.append(image)
                labels.append(label)
        return np.array(images), np.array(labels)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        if self.transform:
            image = self.transform(image)
        return image, label

In [9]:
# Define the transformation to normalize and convert image to tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [11]:
# Load the fine-tuned CLIP model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

In [12]:
# Step 8: Model Training for Nature Classification
dataset = ImageDataset(cnn_data_folder, transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [21]:
# Define the classification head
num_classes = len(set(dataset.labels))
classifier_head = nn.Linear(512, num_classes).to(device)

In [23]:
# Define the optimizer and loss function
optimizer = optim.Adam(classifier_head.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [31]:
# Train the classification head
num_epochs = 100
for epoch in range(num_epochs):
    classifier_head.train()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            image_features = model.encode_image(images)

        # Convert the data type of image_features to match the classifier's weight matrix
        image_features = image_features.to(classifier_head.weight.dtype)

        optimizer.zero_grad()
        logits = classifier_head(image_features)

        # Convert logits data type to torch.float32 for softmax
        logits = logits.to(torch.float32)

        loss = criterion(logits, labels)  # No need to convert labels to torch.long data type
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted_labels = logits.max(1)
        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

    # Calculate accuracy
    accuracy = total_correct / total_samples

    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {total_loss:.4f} - Accuracy: {accuracy:.4f}")


Epoch [1/100] - Loss: 13.2779 - Accuracy: 0.9145
Epoch [2/100] - Loss: 13.3903 - Accuracy: 0.9135
Epoch [3/100] - Loss: 13.3258 - Accuracy: 0.9195
Epoch [4/100] - Loss: 13.2284 - Accuracy: 0.9150
Epoch [5/100] - Loss: 13.1805 - Accuracy: 0.9170
Epoch [6/100] - Loss: 13.1766 - Accuracy: 0.9160
Epoch [7/100] - Loss: 13.1125 - Accuracy: 0.9200
Epoch [8/100] - Loss: 13.0618 - Accuracy: 0.9205
Epoch [9/100] - Loss: 13.0584 - Accuracy: 0.9165
Epoch [10/100] - Loss: 12.9619 - Accuracy: 0.9180
Epoch [11/100] - Loss: 13.0173 - Accuracy: 0.9190
Epoch [12/100] - Loss: 12.8627 - Accuracy: 0.9210
Epoch [13/100] - Loss: 12.9591 - Accuracy: 0.9210
Epoch [14/100] - Loss: 12.8907 - Accuracy: 0.9195
Epoch [15/100] - Loss: 12.7997 - Accuracy: 0.9205
Epoch [16/100] - Loss: 12.8200 - Accuracy: 0.9190
Epoch [17/100] - Loss: 12.7412 - Accuracy: 0.9225
Epoch [18/100] - Loss: 12.8058 - Accuracy: 0.9210
Epoch [19/100] - Loss: 12.6636 - Accuracy: 0.9235
Epoch [20/100] - Loss: 12.6906 - Accuracy: 0.9175
Epoch [21

In [32]:
# Step 9: Model Evaluation for Nature Classification
nature_pred = []
nature_labels = []
classifier_head.eval()
with torch.no_grad():
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        image_features = model.encode_image(images)

        # Convert the data type of image_features to match the classifier's weight matrix
        image_features = image_features.to(classifier_head.weight.dtype)

        logits = classifier_head(image_features)
        pred = logits.argmax(dim=-1).cpu().numpy()
        nature_pred.extend(pred)
        nature_labels.extend(labels.cpu().numpy())

nature_accuracy = accuracy_score(nature_labels, nature_pred)
nature_precision = precision_score(nature_labels, nature_pred, average='weighted')
nature_recall = recall_score(nature_labels, nature_pred, average='weighted')
nature_f1_score = f1_score(nature_labels, nature_pred, average='weighted')

print("Nature Accuracy:", nature_accuracy)
print("Nature Precision:", nature_precision)
print("Nature Recall:", nature_recall)
print("Nature F1-score:", nature_f1_score)


Nature Accuracy: 0.9345
Nature Precision: 0.9347717323327079
Nature Recall: 0.9345
Nature F1-score: 0.934489764025629


In [33]:
# Step 10: Save Nature Model
# Save the fine-tuned classifier head
nature_model_file = 'nature_model.pt'
torch.save(classifier_head.state_dict(), nature_model_file)
print("Nature Model saved successfully.")

Nature Model saved successfully.


In [36]:
import cv2
import torch
import torch.nn as nn
import clip
import numpy as np
from torchvision import transforms

# Load the fine-tuned CLIP model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Define the classification head
num_classes = 2  # Replace with the number of classes in your dataset
classifier_head = nn.Linear(512, num_classes).to(device)
classifier_head.load_state_dict(torch.load('nature_model.pt'))
classifier_head.eval()

# Define the mapping between class indices and class labels
class_mapping = {
    0: "Islamophobic Image",
    1: "Non Islamophobic Image",
}

# Define the transformation to normalize and convert image to tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Function to predict the class of an image
def predict_class(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))  # Resize image to fit CLIP model input shape
    image = transform(image).unsqueeze(0).to(device)  # Convert image to tensor and add batch dimension

    with torch.no_grad():
        image_features = model.encode_image(image)

    # Convert the data type of image_features to match the classifier's weight matrix
    image_features = image_features.to(classifier_head.weight.dtype)

    logits = classifier_head(image_features)
    probabilities = nn.functional.softmax(logits, dim=1)
    predicted_class_index = torch.argmax(probabilities, dim=1).item()
    predicted_class_label = class_mapping[predicted_class_index]

    return predicted_class_label

# Test the prediction function
image_path = 'cnn_dataset/Islamophobic image/islamophobic (1).jpg'
predicted_class = predict_class(image_path)

print(f"Predicted Class: {predicted_class}")


Predicted Class: Islamophobic Image
